In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import urllib.request
import requests
import re
import json as JSON
import sqlite3
from sqlite3 import Error
import sys

In [171]:
gameDate, startTime, attendance, stadium, gameDur = " " * 5
teams = { 'CHC': 'CHN',
            'CHW': 'CHA',
            'KCR': 'KCA',
            'LAA': 'ANA',
            'LAD': 'LAN',
            'NYM': 'NYN',
            'SDP': 'SDN',
            'SFG': 'SFN',
            'STL': 'SLN',
            'TBR': 'TBA',
            'WAS': 'WSN',}

battingNames = {"ATL":"AtlantaBravesbatting",
                    "ARI":"ArizonaDiamondbacksbatting",
                    "BAL":"BaltimoreOriolesbatting",
                    "BOS":"BostonRedSoxbatting",
                    "CHN":"ChicagoCubsbatting",
                    "CHA":"ChicagoWhiteSoxbatting",
                    "CIN":"CincinnatiRedsbatting",
                    "CLE":"ClevelandGuardiansbatting",
                    "COL":"ColoradoRockiesbatting",
                    "DET":"DetroitTigersbatting",
                    "KCA":"KansasCityRoyalsbatting",
                    "HOU":"HoustonAstrosbatting",
                    "ANA":"AnaheimAngelsbatting",
                    "LAN":"LosAngelesDodgersbatting",
                    "MIA":"MiamiMarlinsbatting",
                    "MIL":"MilwaukeeBrewersbatting",
                    "MIN":"MinnesotaTwinsbatting",
                    "NYN":"NewYorkMetsbatting",
                    "NYY":"NewYorkYankeesbatting",
                    "OAK":"OaklandAthleticsbatting",
                    "PHI":"PhiladelphiaPhilliesbatting",
                    "PIT":"PittsburghPiratesbatting",
                    "SDN":"SanDiegoPadresbatting",
                    "SEA":"SeattleMarinersbatting",
                    "SFN":"SanFranciscoGiantsbatting",
                    "SLN":"StLouisCardinalsbatting",
                    "TBA":"TampaBayRaysbatting",
                    "TEX":"TexasRangersbatting",
                    "TOR":"TorontoBlueJaysbatting",
                    "WSN":"WashingtonNationalsbatting"}

pitchingNames = {"ATL":"AtlantaBravespitching",
                    "ARI":"ArizonaDiamondbackspitching",
                    "BAL":"BaltimoreOriolespitching",
                    "BOS":"BostonRedSoxpitching",
                    "CHN":"ChicagoCubspitching",
                    "CHA":"ChicagoWhiteSoxpitching",
                    "CIN":"CincinnatiRedspitching",
                    "CLE":"ClevelandGuardianspitching",
                    "COL":"ColoradoRockiespitching",
                    "DET":"DetroitTigerspitching",
                    "KCA":"KansasCityRoyalspitching",
                    "HOU":"HoustonAstrospitching",
                    "ANA":"AnaheimAngelspitching",
                    "LAN":"LosAngelesDodgerspitching",
                    "MIA":"MiamiMarlinspitching",
                    "MIL":"MilwaukeeBrewerspitching",
                    "MIN":"MinnesotaTwinspitching",
                    "NYN":"NewYorkMetspitching",
                    "NYY":"NewYorkYankeespitching",
                    "OAK":"OaklandAthleticspitching",
                    "PHI":"PhiladelphiaPhilliespitching",
                    "PIT":"PittsburghPiratespitching",
                    "SDN":"SanDiegoPadrespitching",
                    "SEA":"SeattleMarinerspitching",
                    "SFN":"SanFranciscoGiantspitching",
                    "SLN":"StLouisCardinalspitching",
                    "TBA":"TampaBayRayspitching",
                    "TEX":"TexasRangerspitching",
                    "TOR":"TorontoBlueJayspitching",
                    "WSN":"WashingtonNationalspitching"}

In [196]:
hteam = 'MIA'
gameDate = '20220924'
homeTeam=hteam
if homeTeam in teams.keys():
    homeTeam = teams[homeTeam]

In [197]:
url = 'https://www.baseball-reference.com/boxes/%s/%s%s0.shtml' % (homeTeam,homeTeam,gameDate)
res = requests.get(url)
comm = re.compile("<!--|-->")
soup = BeautifulSoup(comm.sub("", res.text), 'html.parser')

scorebox = soup.find(attrs = {'class' : 'scorebox'})
teamLinks=scorebox.select("a[href*=teams]")
ateam = (teamLinks[0]['href'].split('/')[2])
awayTeam=ateam
if awayTeam in teams.keys():
    awayTeam = teams[awayTeam]

print()
print('    ' + hteam + ' vs ' + ateam)
print('------------------------')

gameMetadata = soup.find(attrs = {'class' : 'scorebox_meta'})
gameMetadataList = gameMetadata.find_all('div')
gameDate = gameMetadataList[0].text
startTime = gameMetadataList[1].text
attendance = gameMetadataList[2].text
stadium = gameMetadataList[3].text
gameDur = gameMetadataList[4].text
#startTime = divs[1].text.strip("Start Time: ").replace(' Local','')
#attendance = int(divs[2].text.strip("Attendance: ").replace(',',''))
#stadium = divs[3].text.strip("Venue: ")
#gameDur = divs[4].text.strip("Game Duration: ")
gameInfo = [gameDate, startTime, attendance, stadium, gameDur]
for i in gameInfo:
    print(i)


    MIA vs WSN
------------------------
Saturday, September 24, 2022
Start Time: 6:10 p.m. Local
Attendance: 16,099
Venue: loanDepot park
Game Duration: 2:18


In [198]:
print()
print('------------------------------------------------------------')
print('                         ' + hteam + ' Batting')
print('------------------------------------------------------------')
homeBattingTable = soup.find_all("table", {"id":"%s" % battingNames[homeTeam]})
data_rows = homeBattingTable[0].findAll('tr')
data_header = homeBattingTable[0].findAll('thead')
data_header = data_header[0].findAll("tr")
data_header = data_header[0].findAll("th")
homeBattingRows = [[td.getText() for td in data_rows[i].findAll(['th','td'])]
    for i in range(len(data_rows))
    ]
homeBattingStats = pd.DataFrame(homeBattingRows)
header = []
for i in range(len(homeBattingStats.columns)):
    header.append(data_header[i].getText())
homeBattingStats.columns = header
homeBattingStats = homeBattingStats.loc[homeBattingStats[header[0]] != header[0]].reset_index(drop = True)
homeBattingStats = homeBattingStats[homeBattingStats.PA != ''].reset_index(drop = True)
homeBattingStats.insert(4,'HR',0)
for i in range (len(homeBattingStats)):
    if i == len(homeBattingStats)-1:
        homeBattingStats.at[i,'HR']=homeBattingStats['HR'].sum()
    else:
        for detail in homeBattingStats['Details'][i].split(','):
            if 'HR' in detail:
                if len(detail.split('·')) > 1:
                    homeBattingStats.at[i,'HR']=int(detail.split('·')[0])
                else:
                    homeBattingStats.at[i,'HR']=1
                break
                
homeBattingStats.insert(1,'Pos','N/A')
for i in range (len(homeBattingStats)-1):
    homeBattingStats.at[i, 'Pos'] = homeBattingStats['Batting'][i].split()[2]
    homeBattingStats.at[i, 'Batting'] = homeBattingStats['Batting'][i].split()[0] + ' ' + homeBattingStats['Batting'][i].split()[1]

homeBattingStats.drop(columns=['BA','OBP','SLG','OPS','Pit','Str','WPA','WPA+','WPA-','cWPA','aLI','acLI','RE24','PO','A','Details'],inplace=True)
homeBattingStats


------------------------------------------------------------
                         MIA Batting
------------------------------------------------------------


,Batting,Pos,AB,R,H,HR,RBI,BB,SO,PA
0,Jon Berti,3B,4,0,0,0,0,0,0,4
1,Brian Anderson,RF,4,1,2,0,0,0,0,4
2,Garrett Cooper,DH,3,1,0,0,0,1,3,4
3,Charles Leblanc,2B,4,0,0,0,0,0,1,4
4,JJ Bleday,LF,4,0,1,0,2,0,1,4
5,Bryan De,La,4,2,4,1,1,0,0,4
6,Jacob Stallings,C,4,0,2,0,0,0,1,4
7,Miguel Rojas,SS,3,0,1,0,1,0,0,3
8,Lewin Diaz,1B,3,0,0,0,0,0,2,3
9,Team Totals,N/A,33,4,10,1,4,1,8,34


In [199]:
print()
print('------------------------------------------------------------')
print('                         ' + ateam + ' Batting')
print('------------------------------------------------------------')
awayBattingTable = soup.find_all("table", {"id":"%s" % battingNames[awayTeam]})
data_rows = awayBattingTable[0].findAll('tr')
data_header = awayBattingTable[0].findAll('thead')
data_header = data_header[0].findAll("tr")
data_header = data_header[0].findAll("th")
awayBattingRows = [[td.getText() for td in data_rows[i].findAll(['th','td'])]
    for i in range(len(data_rows))
    ]
awayBattingStats = pd.DataFrame(awayBattingRows)
header = []
for i in range(len(awayBattingStats.columns)):
    header.append(data_header[i].getText())
awayBattingStats.columns = header
awayBattingStats = awayBattingStats.loc[awayBattingStats[header[0]] != header[0]].reset_index(drop = True)
awayBattingStats = awayBattingStats[awayBattingStats.PA != ''].reset_index(drop = True)
awayBattingStats.insert(4,'HR',0)
for i in range (len(awayBattingStats)):
    if i == len(awayBattingStats)-1:
        awayBattingStats.at[i,'HR']=awayBattingStats['HR'].sum()
    else:
        for detail in awayBattingStats['Details'][i].split(','):
            if 'HR' in detail:
                if len(detail.split('·')) > 1:
                    awayBattingStats.at[i,'HR']=int(detail.split('·')[0])
                else:
                    awayBattingStats.at[i,'HR']=1
                break
                
awayBattingStats.insert(1,'Pos','N/A')
for i in range (len(awayBattingStats)-1):
    awayBattingStats.at[i, 'Pos'] = awayBattingStats['Batting'][i].split()[2]
    awayBattingStats.at[i, 'Batting'] = awayBattingStats['Batting'][i].split()[0] + ' ' + awayBattingStats['Batting'][i].split()[1]
awayBattingStats.drop(columns=['BA','OBP','SLG','OPS','Pit','Str','WPA','WPA+','WPA-','cWPA','aLI','acLI','RE24','PO','A','Details'],inplace=True)
awayBattingStats


------------------------------------------------------------
                         WSN Batting
------------------------------------------------------------


KeyError: 'WSN'

In [176]:
print()
print('------------------------------------------------------------------------')
print('                         ' + hteam + ' Pitching')
print('------------------------------------------------------------------------')
homePitchingTable = soup.find_all("table", {"id":"%s" % pitchingNames[homeTeam]})
data_rows = homePitchingTable[0].findAll('tr')
data_header = homePitchingTable[0].findAll('thead')
data_header = data_header[0].findAll("tr")
data_header = data_header[0].findAll("th")
homePitchingRows = [[td.getText() for td in data_rows[i].findAll(['th','td'])]
    for i in range(len(data_rows))
    ]
homePitchingStats = pd.DataFrame(homePitchingRows)
header = []
for i in range(len(homePitchingStats.columns)):
    header.append(data_header[i].getText())
homePitchingStats.columns = header
homePitchingStats = homePitchingStats.loc[homePitchingStats[header[0]] != header[0]]
homePitchingStats.drop(columns=['Ctct','StS','StL','GB','FB','LD','Unk','GSc','IR','IS','WPA','aLI','cWPA','acLI','RE24'],inplace=True)
homePitchingStats = homePitchingStats.reset_index(drop = True)
for i in range (len(homePitchingStats)):
    homePitchingStats.at[i, 'Pitching'] = homePitchingStats['Pitching'][i].split(',')[0]
homePitchingStats


------------------------------------------------------------------------
                         SDP Pitching
------------------------------------------------------------------------


,Pitching,IP,H,R,ER,BB,SO,HR,ERA,BF,Pit,Str
0,Mike Clevinger,6,3,1,1,0,3,1,4.33,21,66,48
1,Robert Suarez,1,0,0,0,0,2,0,2.31,3,16,9
2,Pierce Johnson,0.2,2,1,1,1,0,0,4.73,4,20,11
3,Nick Martinez,0.1,0,0,0,0,0,0,3.50,0,6,3
4,Josh Hader,1,1,0,0,0,2,0,5.29,4,13,9
5,Team Totals,9,6,2,2,1,7,1,2.00,32,121,80


In [177]:
print()
print('------------------------------------------------------------------------')
print('                         ' + ateam + ' Pitching')
print('------------------------------------------------------------------------')
awayPitchingTable = soup.find_all("table", {"id":"%s" % pitchingNames[awayTeam]})
data_rows = awayPitchingTable[0].findAll('tr')
data_header = awayPitchingTable[0].findAll('thead')
data_header = data_header[0].findAll("tr")
data_header = data_header[0].findAll("th")
awayPitchingRows = [[td.getText() for td in data_rows[i].findAll(['th','td'])]
    for i in range(len(data_rows))
    ]
awayPitchingStats = pd.DataFrame(awayPitchingRows)
header = []
for i in range(len(awayPitchingStats.columns)):
    header.append(data_header[i].getText())
awayPitchingStats.columns = header
awayPitchingStats = awayPitchingStats.loc[awayPitchingStats[header[0]] != header[0]]
awayPitchingStats.drop(columns=['Ctct','StS','StL','GB','FB','LD','Unk','GSc','IR','IS','WPA','aLI','cWPA','acLI','RE24'],inplace=True)
awayPitchingStats = awayPitchingStats.reset_index(drop = True)
for i in range (len(awayPitchingStats)):
    awayPitchingStats.at[i, 'Pitching'] = awayPitchingStats['Pitching'][i].split(',')[0]
awayPitchingStats


------------------------------------------------------------------------
                         CHW Pitching
------------------------------------------------------------------------


,Pitching,IP,H,R,ER,BB,SO,HR,ERA,BF,Pit,Str
0,Dylan Cease,5,5,4,4,4,5,2,2.20,24,104,63
1,Joe Kelly,1,2,0,0,0,2,0,6.08,4,13,9
2,Jake Diekman,1,2,1,1,0,1,1,5.17,5,21,16
3,Vince Velasquez,1,0,0,0,1,1,0,4.88,4,14,9
4,Team Totals,8,9,5,5,5,9,3,5.62,37,152,97


In [178]:
homeBattingStats

,Batting,AB,R,H,HR,RBI,BB,SO,PA
0,Jurickson Profar LF,4,0,1,0,0,0,1,4
1,Trent Grisham CF,1,0,0,0,0,0,1,1
2,Juan Soto RF,4,1,1,1,1,0,2,4
3,Manny Machado 3B,4,2,2,1,1,0,2,4
4,Jake Cronenworth 2B,3,1,1,1,2,1,0,4
5,Brandon Drury 1B,3,1,1,0,0,1,1,4
6,Josh Bell DH,3,0,1,0,0,1,0,4
7,Ha-Seong Kim SS,2,0,0,0,0,2,1,4
8,Austin Nola C,4,0,1,0,1,0,1,4
9,Jose Azocar CF-LF,4,0,1,0,0,0,0,4


In [179]:
awayBattingStats

,Batting,AB,R,H,HR,RBI,BB,SO,PA
0,Elvis Andrus SS,4,1,3,1,2,0,0,4
1,Mark Payton LF,3,0,0,0,0,0,1,3
2,Adam Engel PH,1,0,0,0,0,0,0,1
3,Eloy Jiménez DH,4,0,1,0,0,0,0,4
4,Gavin Sheets RF,3,0,0,0,0,0,1,3
5,Romy Gonzalez PH,1,0,0,0,0,0,1,1
6,Yoan Moncada 3B,4,0,0,0,0,0,3,4
7,Andrew Vaughn 1B,3,0,0,0,0,0,1,3
8,AJ Pollock CF,2,1,0,0,0,1,0,3
9,Yasmani Grandal C,3,0,1,0,0,0,0,3


In [180]:
homePitchingStats

,Pitching,IP,H,R,ER,BB,SO,HR,ERA,BF,Pit,Str
0,Mike Clevinger,6,3,1,1,0,3,1,4.33,21,66,48
1,Robert Suarez,1,0,0,0,0,2,0,2.31,3,16,9
2,Pierce Johnson,0.2,2,1,1,1,0,0,4.73,4,20,11
3,Nick Martinez,0.1,0,0,0,0,0,0,3.50,0,6,3
4,Josh Hader,1,1,0,0,0,2,0,5.29,4,13,9
5,Team Totals,9,6,2,2,1,7,1,2.00,32,121,80


In [181]:
awayPitchingStats

,Pitching,IP,H,R,ER,BB,SO,HR,ERA,BF,Pit,Str
0,Dylan Cease,5,5,4,4,4,5,2,2.20,24,104,63
1,Joe Kelly,1,2,0,0,0,2,0,6.08,4,13,9
2,Jake Diekman,1,2,1,1,0,1,1,5.17,5,21,16
3,Vince Velasquez,1,0,0,0,1,1,0,4.88,4,14,9
4,Team Totals,8,9,5,5,5,9,3,5.62,37,152,97
